<center> <h1> Simulación con dinámica molecular de un gas con potencial Lennard-Jones </h1> </center>

## Disclaimer

Parte del código ha sido creado empleando la ayuda de diferentes asistentes de IA como GitHub Copylot, ChatGPT o DeepSeek si bien el análisis ha sido completamente genuino.

## Introducción

Un potencial de tipo Lennard-Jones se usa para caracterizar sistemas en los que la interacción entre partículas es atractiva a distancias largas y altamente repulsiva cuando esta se encuentran muy cerca. El uso de este tipo de potencial para simular sistemas de partículas nos es útil para poder crear modelos de sistemas de partículas con interacciones microscópicas como átomos o moléculas sin que sea muy costoso a nivel computacional.

El potencial de Lennard-Jones viene descrito mediante la siguiente ecuación
$$
V(r) = 4 \epsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^6 \right]
$$

Donde $\epsilon$ determina la profundidad del potencial y $\sigma$ está relacionada con la posición del mínimo. La primera imagen muestra un potencial para escala de energía genérica $\epsilon$ y distancia de interacción genérica $\sigma$. Cuando se disponen las partículas del sistema de forma organizada y separada entre sí una distancia $r = 2 ^(1/6) \sim 1.22\sigma$ estas caen en un pozo de potencial que en primera aproximación y para desplazamientos pequeños es armónico, como se muestra en la segunda imagen

<div style="display: flex;">
<img src="Animaciones y graficas/Potencial.png" width="500" style="margin-right: 20px">
<img src="Animaciones y graficas/Potencial simetrico.png" width="500">
</div>


En la primera imagen la asíntota en $ r = 0 $  simula un núcleo duro o una fuerza fuerte que repele entre sí a las partículas cuando se acercan demasiado haciendo que reboten entre sí. 



<div style="display: flex;">
<img src="Animaciones y graficas/Condiciones_periodicas.PNG" width=400 style="margin-right: 12px;">
Este proyecto consiste en la simulación de un conjunto de partículas interaccionando entre sí mediante un potencial de tipo Lennard-Jones encerradas en una caja y sometidas a condiciones de contorno periódicas aplicando condición de imagen mínima para la posición de las partículas.<br><br>
Cuando una partícula atraviesa una de las paredes aparece en la pared opuesta con la misma dirección y sentido en la velocidad que seguiría tras atravesar la pared. Además al calcular las distancias entre partículas se asume que detrás de cada pared hay una copia exacta de todas las partículas en la caja, por lo que si dos partículas se encuentran a una distacia mayor a $ L/2 $ se le resta $ L/2 $ para poder representar una red o conjunto virtualmente infinito de partículas trabajando con tan solo unas pocas. <br><br> Se empleará un algoritmo de Verlet de un solo paso (Verlet-Velocity) para hacer el cálculo basado en el cálculo de la posición de cada partícula en cada paso calculando la aceleración que sufre cada partícula debida a la interacción con el resto
</div>

## Desarrollo del programa

### Gestión del proyecto

La simulación se ha llevado a cabo en C++17 y el análisis de datos en Python 3.12.8 . Todos los programas, animaciones y gráficas se puedne encontrar en el repositorio de GitHub en la carpeta designada
- Compu/
    - Lennard_Jones/
        - Analisis/
        - Animaciones y graficas/
        - Simulacion/
            - Datos/
            - lennard-jones-lib/
            - ...

## Análisis estadístico de la velocidad

### Inicio con velocidades aleatorias

Para un sistema de partículas clásicas que interaccionan entre sí a corta distancia intercambiando momento y energía mediante choques elásticos se puede demostrar que si las velocidades sobre los ejes de las partículas vienen dadas por una distribución normal, $N(\mu, \sigma)$, la el módulo de la velocidad de las partículas del sistema sigue una distribución del tipo Maxwell-Boltzmann de dimensión $d$, la dimensión del sistema, 2D en este caso.

Para comrpobar si el potencial Lennard-Jones genera un sistema en el que se cumpla la ecuación de Boltzmann, y por tanto que evolucione a un estado en el que las velocidades siguen una distribución Maxwell-Boltzmann se ha inicializado el sistema con las velocidades de las partículas configuradas de la siguiente forma.

- Se selecciona un valor para $|v|$ idéntico para todas las partículas
- Se selecciona la dirección de la velocidad mediante una distribución uniforme entre $0$ y $2\pi$

Sin embargo por el hecho de que $\theta$ venga dada por una distribución uniforme entre $0$ y $2\pi$, $v_x = v \cos(\theta)$ y $v_y = v \sin(\theta)$, se observa que las velocidades sobre los ejes, $v_x$ y $v_y$, no siguen una distrubución normal, sino una distribución arcoseno generalizada.

![PDF velocidades sobre ejes](Animaciones%20y%20graficas/PDF%20arcoseno.png)

Por lo que el estudio de unas pocas partículas no mostraría el resultado esperado, sin embargo empleando el teorema central del límite si tomamos como datos las velocidades de las partículas desde un tiempo $t = 20$, cuando el sistema ya ha alcanzado el *equilibrio*, hasta $t = 50$ o $t = 2000$ se observa como las componentes de los ejes de la velocidad convergen a una distribución normal de media $\mu = 0$ y varianza $\sigma^2 = T k_B/m$

$$
P(v_x) = \sqrt{\frac{m}{2\pi k_BT}} \; e^{-\frac{m{v_x}^2}{2k_BT}}
$$

Haciendo uso de herramientas de estadística se demuestra que al distribución de probabilidad que sigue el módulo de la velocidad es Maxwell-Boltzmann bidimensional dada por el parámetro $a = T k_B/m$.

$$
P(v) = \left(\frac{m}{k_BT}\right)ve^{-\frac{mv^2}{2k_BT}}
$$

Los parámetros estadísticos de estas distribuciones vienen dados por la temperatura del sistema, que aplicando la definición de temperatura dada por la teoría cinética de los gases es

$$
k_B T = \langle E_{cinetica} \rangle = \frac{1}{2} m \langle v^2 \rangle \Rightarrow T = \frac{m}{2k_B} \langle v^2 \rangle
$$

Para simplificar los cálculos y las unidades se toma $m = 1$ y $k_B = 1$. La siguiente figura muestra un histograma de la distribución de probabilidad de las velocidades entre los tiempos $t = 20-50$ donde se observa claramente que se ajusta a la distribución teórica, tanto para $v$ como para $v_x$, se ha seleccionado $|v| = 10$ para observar la animación claramente.

<img src="Animaciones%20y%20graficas/Distribuciones%20de%20probabilidad%2025000%20pasos%20y%2016%20partículas.png" width=800 style="width: 95%;">

La siguiente animación muestra la evolución en la distribución de las velocidades según el sistema alcanza el equilibrio

<video width="800" controls>
  <source src="Animaciones%20y%20graficas/Animacion velocidades 100 partículas y 5000 pasos.mp4" type="video/mp4">
</video>

### Inicio decompensado: $v_x > v_y$

## Ecuación de estado

## Transición sólido-líquido

### Estudio de estabilidad del sólido

### Estudio de la fusión: cambio de estado